## 2026 EY AI & Data Challenge - TerraClimate Data Extraction Notebook

This notebooks demonstrates how to access the TerraClimate dataset. TerraClimate is a dataset of monthly climate and climatic water balance for global terrestrial surfaces from 1958 to the present. These data provide important inputs for ecological and hydrological studies at global scales that require high spatial resolution and time-varying data. All data have monthly temporal resolution and a ~4-km (1/24th degree) spatial resolution. This dataset is provided in Zarr format. 

For more information, visit: https://planetarycomputer.microsoft.com/dataset/terraclimate#overview 

In [37]:
import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

from scipy.spatial import cKDTree

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc

from datetime import date
from tqdm import tqdm
import os

<h2>Extracting TerraClimate Data Using API Calls</h2> <p align="justify"> The API-based method allows us to efficiently access <b>TerraClimate</b> data for specific regions and time periods through the <a href="https://planetarycomputer.microsoft.com/">Microsoft Planetary Computer</a>, ensuring scalability and reproducibility of the process. </p> <p align="justify"> Through the API, we can extract climate variables such as <b>Potential Evapotranspiration (PET)</b>, which represents the atmospheric demand for water. This variable provides critical insights into surface moisture balance and helps improve the accuracy of water quality modeling. </p> <p align="justify"> This approach ensures consistent, automated retrieval of high-resolution climate data that can be easily integrated with satellite-derived features for comprehensive environmental and hydrological analysis. </p>



<h3>Loading and Mapping TerraClimate Data:</h3>

<p>This section demonstrates how <b>TerraClimate climate variables</b>, such as <b>Potential Evapotranspiration (PET)</b>, are loaded and mapped to sampling locations:</p>

<ul>
  <li>The <b>load_terraclimate_dataset</b> function opens the TerraClimate Zarr/NetCDF dataset from the Microsoft Planetary Computer, handling storage options automatically.</li>
  <li>The <b>filterg</b> function filters the dataset for the desired time range (2011–2015) and spatial extent corresponding to the study region. The resulting data is converted to a pandas DataFrame with standardized column names.</li>
  <li>The <b>assign_nearest_climate</b> function maps each sampling location to its <b>nearest TerraClimate grid point</b> using a KD-tree and assigns the climate variable values corresponding to the closest time stamp.</li>
</ul>

<p>This workflow ensures efficient, reproducible retrieval of climate variables, while allowing participants to work with pre-extracted CSV files for faster benchmarking and analysis.</p>


In [38]:
def load_terraclimate_dataset():
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
    )
    
    collection = catalog.get_collection("terraclimate")
    asset = collection.assets["zarr-abfs"]
    
    signed_asset = pc.sign(asset)

    if "xarray:storage_options" in signed_asset.extra_fields:
        ds = xr.open_zarr(
            signed_asset.href,
            storage_options=signed_asset.extra_fields["xarray:storage_options"],
            consolidated=True,
        )
    else:
        ds = xr.open_dataset(
            signed_asset.href,
            **signed_asset.extra_fields["xarray:open_kwargs"],
        )

    return ds

In [39]:
# --- Filtering function (kept identical) ---
def filterg(ds, var):
    print(f"Iniciando recorte otimizado para a variável: {var}...")
    
    # 1. Filtro Direto no xarray (O segredo da performance)
    # Selecionamos o tempo e o recorte espacial enquanto os dados ainda estão em formato de "grade"
    # O xarray é extremamente eficiente nisso.
    
    # Nota: No TerraClimate, a latitude costuma ser decrescente (do Norte para o Sul). 
    # Por isso, usamos o slice do maior valor para o menor no 'lat'.
    ds_subset = ds[var].sel(
        time=slice("2011-01-01", "2015-12-31"),
        lat=slice(-21.72, -35.18), 
        lon=slice(14.97, 32.79)
    )

    # 2. Converte apenas o recorte para DataFrame de uma vez só (sem loops!)
    df_var_final = ds_subset.to_dataframe().reset_index()

    # 3. Formatação final (mantendo compatibilidade com seu código)
    df_var_final['time'] = df_var_final['time'].astype(str)
    col_mapping = {"lat": "Latitude", "lon": "Longitude", "time": "Sample Date"}
    df_var_final = df_var_final.rename(columns=col_mapping)

    print(f"Filtering for {var} completed!")
    return df_var_final


In [40]:
# --- Climate variable assignment function (unchanged logic) ---
def assign_nearest_climate(sa_df, climate_df, var_name):
    """
    Map nearest climate variable values to a new DataFrame 
    containing only the specified variable column.
    """
    sa_coords = np.radians(sa_df[['Latitude', 'Longitude']].values)
    climate_coords = np.radians(climate_df[['Latitude', 'Longitude']].values)

    tree = cKDTree(climate_coords)
    dist, idx = tree.query(sa_coords, k=1)

    nearest_points = climate_df.iloc[idx].reset_index(drop=True)

    sa_df = sa_df.reset_index(drop=True)
    sa_df[['nearest_lat', 'nearest_lon']] = nearest_points[['Latitude', 'Longitude']]

    sa_df['Sample Date'] = pd.to_datetime(sa_df['Sample Date'], dayfirst=True, errors='coerce')
    climate_df['Sample Date'] = pd.to_datetime(climate_df['Sample Date'], dayfirst=True, errors='coerce')

    climate_values = []

    for i in tqdm(range(len(sa_df)), desc=f"Mapping {var_name.upper()} values"):
        sample_date = sa_df.loc[i, 'Sample Date']
        nearest_lat = sa_df.loc[i, 'nearest_lat']
        nearest_lon = sa_df.loc[i, 'nearest_lon']

        subset = climate_df[
            (climate_df['Latitude'] == nearest_lat) &
            (climate_df['Longitude'] == nearest_lon)
        ]

        if subset.empty:
            climate_values.append(np.nan)
            continue

        nearest_idx = (subset['Sample Date'] - sample_date).abs().idxmin()
        climate_values.append(subset.loc[nearest_idx, var_name])

    output_df = pd.DataFrame({var_name: climate_values})

    
    return output_df

### Extracting features for the training dataset

In [12]:
Water_Quality_df=pd.read_csv('water_quality_training_dataset.csv')
Water_Quality_df_100 = Water_Quality_df[0:100]
Water_Quality_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [13]:
Water_Quality_df.shape

(9319, 6)

In [30]:
# Load TerraClimate dataset, filter (time,region,parameter), filter for nearest parameter values
ds = load_terraclimate_dataset()
tc_parameter = filterg(ds,'pet')
Terraclimate_training_df = assign_nearest_climate(Water_Quality_df_100, tc_parameter, 'pet')
Terraclimate_training_df

Iniciando recorte otimizado para a variável: pet...
Filtering for pet completed!


Mapping PET values: 100%|██████████| 100/100 [00:01<00:00, 61.82it/s]


,pet
0,174.199997
1,124.099998
2,127.500000
3,129.699997
4,129.199997
...,...
95,164.100006
96,179.300003
97,154.600006
98,170.900009


In [42]:
# testing all bands of terraclimate 100 df
ds=load_terraclimate_dataset()

features_dataframe = Water_Quality_df[['Latitude', 'Longitude', 'Sample Date']].copy()

for var in ['q', 'ppt', 'pet', 'soil', 'pdsi', 'tmax', 'tmin']:
    df_clima = filterg(ds, var)
    coluna_result = assign_nearest_climate(Water_Quality_df, df_clima, var)
    features_dataframe[var] = coluna_result[var]

Iniciando recorte otimizado para a variável: q...
Filtering for q completed!


Mapping Q values: 100%|██████████| 9319/9319 [02:31<00:00, 61.31it/s]


Iniciando recorte otimizado para a variável: ppt...
Filtering for ppt completed!


Mapping PPT values: 100%|██████████| 9319/9319 [02:33<00:00, 60.84it/s]


Iniciando recorte otimizado para a variável: pet...
Filtering for pet completed!


...

Mapping PDSI values: 100%|██████████| 9319/9319 [02:20<00:00, 66.21it/s]


Iniciando recorte otimizado para a variável: tmax...
Filtering for tmax completed!


Mapping TMAX values: 100%|██████████| 9319/9319 [02:33<00:00, 60.75it/s]


Iniciando recorte otimizado para a variável: tmin...
Filtering for tmin completed!


Mapping TMIN values: 100%|██████████| 9319/9319 [02:33<00:00, 60.65it/s]


In [43]:
Terraclimate_training_df = features_dataframe.copy()
features_dataframe

,Latitude,Longitude,Sample Date,q,ppt,pet,soil,pdsi,tmax,tmin
0,-28.760833,17.730278,02-01-2011,1.6,32.7,174.199997,0.0,3.65,36.099998,22.689999
1,-26.861111,28.884722,03-01-2011,2.6,51.1,124.099998,12.8,0.66,27.160000,13.219999
2,-26.450000,28.085833,03-01-2011,3.1,62.7,127.500000,6.8,-1.16,27.519999,14.090000
3,-27.671111,27.236944,03-01-2011,4.2,84.2,129.699997,7.2,2.84,28.869999,14.639999
4,-27.356667,27.286389,03-01-2011,3.9,78.0,129.199997,7.8,2.65,28.670000,14.690000
...,...,...,...,...,...,...,...,...,...,...
9314,-27.527500,30.858056,23-12-2015,1.5,29.9,166.300003,5.3,-3.84,29.969999,16.400000
9315,-26.861111,28.884722,23-12-2015,1.8,35.8,182.400009,2.2,-4.36,29.830000,14.120000
9316,-26.984722,26.632278,23-12-2015,1.3,26.2,207.800003,0.5,-4.29,32.930000,17.020000
9317,-27.935000,26.126667,23-12-2015,0.6,12.9,222.800003,0.4,-3.95,34.219997,16.250000


In [44]:
Terraclimate_training_df['Latitude'] = Water_Quality_df['Latitude']
Terraclimate_training_df['Longitude'] = Water_Quality_df['Longitude']
Terraclimate_training_df['Sample Date'] = Water_Quality_df['Sample Date']
Terraclimate_training_df = Terraclimate_training_df[['Latitude', 'Longitude', 'Sample Date', 'q', 'ppt', 'pet', 'soil', 'pdsi', 'tmax', 'tmin']]

eps = 1e-10  # Small constant to avoid division by zero
Terraclimate_training_df['Saline_Index'] = (Terraclimate_training_df['pet'] - Terraclimate_training_df['ppt']) / (Terraclimate_training_df['ppt'] + Terraclimate_training_df['pet'] + eps)
Terraclimate_training_df['Runoff_Ratio'] = Terraclimate_training_df['q'] / (Terraclimate_training_df['ppt'] + eps)
Terraclimate_training_df['Chemical_Solubility'] = (Terraclimate_training_df['tmax'] + Terraclimate_training_df['tmin']) / 2



Terraclimate_training_df.to_csv('../Datasets/terraclimate_features_more_bands_training.csv', index=False)

In [45]:
# Preview File
Terraclimate_training_df.head()

,Latitude,Longitude,Sample Date,q,ppt,pet,soil,pdsi,tmax,tmin,Saline_Index,Runoff_Ratio,Chemical_Solubility
0,-28.760833,17.730278,02-01-2011,1.6,32.7,174.199997,0.0,3.65,36.099998,22.689999,0.683905,0.048930,29.394999
1,-26.861111,28.884722,03-01-2011,2.6,51.1,124.099998,12.8,0.66,27.160000,13.219999,0.416667,0.050881,20.189999
2,-26.450000,28.085833,03-01-2011,3.1,62.7,127.500000,6.8,-1.16,27.519999,14.090000,0.340694,0.049442,20.805000
3,-27.671111,27.236944,03-01-2011,4.2,84.2,129.699997,7.2,2.84,28.869999,14.639999,0.212716,0.049881,21.754999
4,-27.356667,27.286389,03-01-2011,3.9,78.0,129.199997,7.8,2.65,28.670000,14.690000,0.247104,0.050000,21.680000


### Extracting features for the validation dataset

In [46]:
Validation_df=pd.read_csv('submission_template.csv')
Validation_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN


In [47]:
Validation_df.shape

(200, 6)

In [48]:
# Load TerraClimate dataset, filter (time,region,parameter), filter for nearest parameter values
Terraclimate_validation_df = pd.DataFrame()
for var in ['q', 'ppt', 'pet', 'soil', 'pdsi', 'tmax', 'tmin']:
    df_clima = filterg(ds, var)
    coluna_result = assign_nearest_climate(Validation_df, df_clima, var)
    Terraclimate_validation_df[var] = coluna_result[var]

Iniciando recorte otimizado para a variável: q...
Filtering for q completed!


Mapping Q values: 100%|██████████| 200/200 [00:03<00:00, 51.76it/s]


Iniciando recorte otimizado para a variável: ppt...
Filtering for ppt completed!


Mapping PPT values: 100%|██████████| 200/200 [00:03<00:00, 60.15it/s]


Iniciando recorte otimizado para a variável: pet...
Filtering for pet completed!


...

Mapping PDSI values: 100%|██████████| 200/200 [00:03<00:00, 60.64it/s]


Iniciando recorte otimizado para a variável: tmax...
Filtering for tmax completed!


Mapping TMAX values: 100%|██████████| 200/200 [00:03<00:00, 60.18it/s]


Iniciando recorte otimizado para a variável: tmin...
Filtering for tmin completed!


Mapping TMIN values: 100%|██████████| 200/200 [00:03<00:00, 60.80it/s]


In [49]:
Terraclimate_validation_df['Latitude'] = Validation_df['Latitude']
Terraclimate_validation_df['Longitude'] = Validation_df['Longitude']
Terraclimate_validation_df['Sample Date'] = Validation_df['Sample Date']

# tem que fazer o calculo dos indices aqui também

eps = 1e-10  # Small constant to avoid division by zero
Terraclimate_validation_df['Saline_Index'] = (Terraclimate_validation_df['pet'] - Terraclimate_validation_df['ppt']) / (Terraclimate_validation_df['ppt'] + Terraclimate_validation_df['pet'] + eps)
Terraclimate_validation_df['Runoff_Ratio'] = Terraclimate_validation_df['q'] / (Terraclimate_validation_df['ppt'] + eps)
Terraclimate_validation_df['Chemical_Solubility'] = (Terraclimate_validation_df['tmax'] + Terraclimate_validation_df['tmin']) / 2

Terraclimate_validation_df = Terraclimate_validation_df[['Latitude', 'Longitude', 'Sample Date', 'q', 'ppt', 'pet', 'soil', 'pdsi', 'tmax', 'tmin']]
Terraclimate_validation_df.to_csv('../Datasets/terraclimate_features_more_bands_validation.csv', index=False)

In [14]:
# Preview File
Terraclimate_validation_df.head()

,Latitude,Longitude,Sample Date,pet
0,-32.043333,27.822778,01-09-2014,161.900009
1,-33.329167,26.077500,16-09-2015,177.600006
2,-32.991639,27.640028,07-05-2015,158.400009
3,-34.096389,24.439167,07-02-2012,130.000000
4,-32.000556,28.581667,01-10-2014,152.500000
